<div dir=rtl align=center>

<img src='https://upload.wikimedia.org/wikipedia/fa/thumb/a/a9/Sharif_logo.svg/626px-Sharif_logo.svg.png?20110526112825' alt="SUT logo" width=200 height=200 align=center  >
<br>
<font face="B Yekan">
<font color=0F5298 size=7>
پروژه تحقیق در عملیات 2<br>
<font color=2565AE size=5>
دانشکده مهندسی صنایع<br>
<font color=2565AE size=4>
دکتر مریم رادمان <br>
<font  size=4>
 <br>
پاییز 1402<br>

<font color=3C99D size=5>
فاز دوم<br>

<br>  
    
    هستی شاه‌حسینی 99104016

    دانیال پیرا 99103739

    امیرحسین حسنی 99103803 
    
    
    
____


### سوال 1 :

نام گذاری تمام متغیرها بر اساس نام گذاری فاز اول انجام شده و بر همان اساس است.

بدین صورت که x_K,x_L,x_M سطح خروجی ژنراتور K,L,Mام نوع 1و2و3 در بازه زمانی j هستند.(K نوع اول، L نوع دوم و M نوع سوم)

متغیرهای Y_K,Y_L,Y_M هم نشان می‌دهند در بازه j ژنراتور مربوطه کار میکند یا خیر.

همچنین S نیز تعداد ژنراتورهای نوع i که در بازهی j راه اندازی میشوند(iهای 1و2و3 همان نوع K  و L و M هستند.)(دو بعد i,j)

و T تعداد ساعت در بازه زمانی j را نشان میدهد که ورودی مساله است

دقت شود که ما برای متغیرها سه بعد قائل شدیم: سه متغیر برای هر نوع ژنراتور. و هر متغیر دو بعد iمین از تعداد ژنراتورهای آن نوع و jمین بازه زمانی را دارد.

#### مدل سازی مساله  با استفاده از کتاب خانه pulp انجام شده. 

In [20]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus, LpMaximize
import pandas as pd

# Define the problem
model = LpProblem(name="GeneratorOptimization", sense=LpMinimize)

# define the number of j,K,L,M's for our varaibles
j_intervals = range(1, 6)
K_generators = range(1, 13)
L_generators = range(1, 11)
M_generators = range(1, 6)
# Define decision variables
S = LpVariable.dicts("S", ((t, j) for t in range(1, 4) for j in j_intervals), lowBound=0, cat="Integer")
x_K = LpVariable.dicts("xK", ((k, j) for k in K_generators for j in j_intervals), lowBound=0, cat="Continuous")
x_L = LpVariable.dicts("xL", ((l, j) for l in L_generators for j in j_intervals), lowBound=0, cat="Continuous")
x_M = LpVariable.dicts("xM", ((m, j) for m in M_generators for j in j_intervals), lowBound=0, cat="Continuous")
Y_K = LpVariable.dicts("yK", ((K, j) for K in K_generators for j in j_intervals), cat="Binary")
Y_L = LpVariable.dicts("yL", ((L, j) for L in L_generators for j in j_intervals), cat="Binary")
Y_M = LpVariable.dicts("yM", ((M, j) for M in M_generators for j in j_intervals), cat="Binary")
T = {1: 6, 2: 3, 3: 6, 4: 3, 5: 6}

# Define objective function from the model 1 of phase1
model += lpSum(
    2 * (lpSum(x_K[k, j] for k in K_generators) - 850 * lpSum(Y_K[K, j] for K in K_generators)) +
    1.3 * (lpSum(x_L[l, j] for l in L_generators) - 1250 * lpSum(Y_L[L, j] for L in L_generators)) +
    3 * (lpSum(x_M[m, j] for m in M_generators) - 1500 * lpSum(Y_M[M, j] for M in M_generators)) for j in j_intervals
) + lpSum(
    1000 * (lpSum(Y_K[K, j]*T[j] for K in K_generators)) +
    2600 * (lpSum(Y_L[L, j]*T[j] for L in L_generators)) +
    3000 * (lpSum(Y_M[M, j]*T[j] for M in M_generators)) for j in j_intervals   
) + lpSum(2000 * S[1, j] + 1000 * S[2, j] + 500 * S[3, j] for j in j_intervals)

## Define constraints

# constraint 1: Demand constraints
model += lpSum(x_K[K, 1] for K in K_generators )+ lpSum(x_L[L, 1] for L in L_generators )+lpSum(x_M[M, 1] for M in M_generators ) >= 15000
model += lpSum(x_K[K, 2] for K in K_generators )+ lpSum(x_L[L, 2] for L in L_generators )+lpSum(x_M[M, 2] for M in M_generators ) >= 30000
model += lpSum(x_K[K, 3] for K in K_generators )+ lpSum(x_L[L, 3] for L in L_generators )+lpSum(x_M[M, 3] for M in M_generators ) >= 25000
model += lpSum(x_K[K, 4] for K in K_generators )+ lpSum(x_L[L, 4] for L in L_generators )+lpSum(x_M[M, 4] for M in M_generators ) >= 40000
model += lpSum(x_K[K, 5] for K in K_generators )+ lpSum(x_L[L, 5] for L in L_generators )+lpSum(x_M[M, 5] for M in M_generators ) >= 27000

# constraint 2: Minimum and maximum range constraints
for j in j_intervals:
    for K in K_generators:
        model += x_K[K, j] >= 850 * Y_K[K, j] 
        model += x_K[K, j] <= 2000 * Y_K[K, j]

    for L in L_generators:
        model += x_L[L, j] >= 1250 * Y_L[L, j] 
        model += x_L[L, j] <= 1750 * Y_L[L, j]

    for M in M_generators:
        model += x_M[M, j] >= 1500 * Y_M[M, j] 
        model += x_M[M, j] <= 4000 * Y_M[M, j]
        
        
# constraint 3: Ability to increase load up to 15%
model += 2000 * lpSum(Y_K[K, 1] for K in K_generators)+ 1750 * lpSum(Y_L[L, 1] for L in L_generators) +4000*lpSum(Y_M[M, 1] for M in M_generators) >= 1.15 * 15000
model += 2000 * lpSum(Y_K[K, 2] for K in K_generators)+ 1750 * lpSum(Y_L[L, 2] for L in L_generators) +4000*lpSum(Y_M[M, 2] for M in M_generators) >= 1.15 * 30000
model += 2000 * lpSum(Y_K[K, 3] for K in K_generators)+ 1750 * lpSum(Y_L[L, 3] for L in L_generators) +4000*lpSum(Y_M[M, 3] for M in M_generators) >= 1.15 * 25000
model += 2000 * lpSum(Y_K[K, 4] for K in K_generators)+ 1750 * lpSum(Y_L[L, 4] for L in L_generators) +4000*lpSum(Y_M[M, 4] for M in M_generators) >= 1.15 * 40000
model += 2000 * lpSum(Y_K[K, 5] for K in K_generators)+ 1750 * lpSum(Y_L[L, 5] for L in L_generators) +4000*lpSum(Y_M[M, 5] for M in M_generators) >= 1.15 * 27000

# constraint 4: The limitation of relationship between the generator start-up variable and the number of working units
for j in j_intervals:
    if j == 1:
        model += S[1, j] >= lpSum(Y_K[K, j] for K in K_generators) - lpSum(Y_K[K, 5] for K in K_generators)
        model += S[2, j] >= lpSum(Y_L[L, j] for L in L_generators) - lpSum(Y_L[L, 5] for L in L_generators)
        model += S[3, j] >= lpSum(Y_M[M, j] for M in M_generators) - lpSum(Y_M[M, 5] for M in M_generators)
    else:
        model += S[1, j] >= lpSum(Y_K[K, j] for K in K_generators) - lpSum(Y_K[K, j-1] for K in K_generators)
        model += S[2, j] >= lpSum(Y_L[L, j] for L in L_generators) - lpSum(Y_L[L, j-1] for L in L_generators)
        model += S[3, j] >= lpSum(Y_M[M, j] for M in M_generators) - lpSum(Y_M[M, j-1] for M in M_generators)

# Solve the problem
model.solve()

# Print the results and the solving status
print(f"Status: {LpStatus[model.status]}, Objective Value: {value(model.objective)}")


# Print decision variable values
for v in model.variables():
    
    print(f"{v.name}: {v.varValue}")
    

Status: Optimal, Objective Value: 580200.0
S_(1,_1): 0.0
S_(1,_2): 3.0
S_(1,_3): 0.0
S_(1,_4): 1.0
S_(1,_5): 0.0
S_(2,_1): 0.0
S_(2,_2): 0.0
S_(2,_3): 0.0
S_(2,_4): 2.0
S_(2,_5): 0.0
S_(3,_1): 0.0
S_(3,_2): 3.0
S_(3,_3): 0.0
S_(3,_4): 3.0
S_(3,_5): 0.0
xK_(1,_1): 0.0
xK_(1,_2): 2000.0
xK_(1,_3): 2000.0
xK_(1,_4): 2000.0
xK_(1,_5): 2000.0
xK_(10,_1): 1300.0
xK_(10,_2): 2000.0
xK_(10,_3): 2000.0
xK_(10,_4): 2000.0
xK_(10,_5): 2000.0
xK_(11,_1): 2000.0
xK_(11,_2): 2000.0
xK_(11,_3): 2000.0
xK_(11,_4): 2000.0
xK_(11,_5): 2000.0
xK_(12,_1): 2000.0
xK_(12,_2): 2000.0
xK_(12,_3): 2000.0
xK_(12,_4): 2000.0
xK_(12,_5): 2000.0
xK_(2,_1): 0.0
xK_(2,_2): 2000.0
xK_(2,_3): 2000.0
xK_(2,_4): 2000.0
xK_(2,_5): 2000.0
xK_(3,_1): 2000.0
xK_(3,_2): 2000.0
xK_(3,_3): 2000.0
xK_(3,_4): 2000.0
xK_(3,_5): 2000.0
xK_(4,_1): 850.0
xK_(4,_2): 2000.0
xK_(4,_3): 2000.0
xK_(4,_4): 2000.0
xK_(4,_5): 2000.0
xK_(5,_1): 0.0
xK_(5,_2): 2000.0
xK_(5,_3): 2000.0
xK_(5,_4): 2000.0
xK_(5,_5): 2000.0
xK_(6,_1): 2000.0
xK_(

برای بهتر نشان دادن متغیرها یک بار آن‌ها را در قابل دیتا فریم ذخیره کردیم و نمایش میدهیم.

In [21]:
# Create an empty dataframe with the desired column names
df_x = pd.DataFrame(columns=["K generator", "L generator", "M generator"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    row_values = {
        "K generator": sum([x_K[k, j].varValue for k in K_generators]),
        "L generator": sum([x_L[l, j].varValue for l in L_generators]),
        "M generator": sum([x_M[m, j].varValue for m in M_generators])
    }
    df_x = df_x.append(row_values, ignore_index=True)
# Set the row index to start from 1
df_x.index = range(1, len(df_x) + 1)
    
# Display the final dataframe
print("The x_K and x_L and x_M in 5 intervals")
print()
print(df_x)


The x_K and x_L and x_M in 5 intervals

   K generator  L generator  M generator
1      15000.0          0.0          0.0
2      24000.0          0.0       6000.0
3      22000.0          0.0       3000.0
4      24000.0       3500.0      12500.0
5      24000.0          0.0       3000.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1875538281.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x = df_x.append(row_values, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1875538281.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x = df_x.append(row_values, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1875538281.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x = df_x.append(row_values, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1875538281.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x = df_x.append(row_values, ignore_inde

In [22]:
# Create an empty dataframe with the desired column names
df_y = pd.DataFrame(columns=["K generator", "L generator", "M generator"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    row_values_y = {
        "K generator": sum([Y_K[k, j].varValue for k in K_generators]),
        "L generator": sum([Y_L[l, j].varValue for l in L_generators]),
        "M generator": sum([Y_M[m, j].varValue for m in M_generators])
    }
    df_y = df_y.append(row_values_y, ignore_index=True)
# Set the row index to start from 1
df_y.index = range(1, len(df_y) + 1)
    
# Display the final dataframe
print("The y_K and y_L and y_M in 5 intervals")
print()
print(df_y)


The y_K and y_L and y_M in 5 intervals

   K generator  L generator  M generator
1          9.0          0.0          0.0
2         12.0          0.0          3.0
3         11.0          0.0          2.0
4         12.0          2.0          5.0
5         12.0          0.0          2.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\2203059812.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y = df_y.append(row_values_y, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\2203059812.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y = df_y.append(row_values_y, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\2203059812.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y = df_y.append(row_values_y, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\2203059812.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y = df_y.append(row_values_y, ign

In [23]:
# Create an empty dataframe with the desired column names
df_S = pd.DataFrame(columns=["K generator", "L generator", "M generator"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    for t in range(1,4):
        row_values_S = {
            "K generator": S[t, j].varValue ,
            "L generator": S[t, j].varValue ,
            "M generator": S[t, j].varValue
    }
    df_S = df_S.append(row_values_S, ignore_index=True)
# Set the row index to start from 1
df_S.index = range(1, len(df_S) + 1)
    
# Display the final dataframe
print("The S for 3 generator types and 5 intervals")
print()
print(df_S)


The S for 3 generator types and 5 intervals

   K generator  L generator  M generator
1          0.0          0.0          0.0
2          3.0          3.0          3.0
3          0.0          0.0          0.0
4          3.0          3.0          3.0
5          0.0          0.0          0.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\3707630358.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S = df_S.append(row_values_S, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\3707630358.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S = df_S.append(row_values_S, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\3707630358.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S = df_S.append(row_values_S, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\3707630358.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S = df_S.append(row_values_S, ign

### سوال :  هزینه حاشیهای تولید برق در هر دوره از روز چقدر است؟

این سوال به این معنی است که اگر ما مقدار برق مورد نیاز در هر بازه زمانی ( که مثلا در بازه زمانی اول 15000 MW بود) را به اندازه‌ی 1 مگاوات در ساعت زیاد کنیم هزینه‌ی نهایی ما(objective function) چقدر تغییر می‌کند.
این به این معناست که مثلا اگر ما در بازه‌ی زمانی اول j=1 محدودیت اول که مربوط به محدودیت تقاضا در بازه‌ی زمانی اول است به اندازه‌ی 1 واحد اضافه میشود و این تغییر در محدودیت ما سبب چه میزان تغییر در جواب بهینه نهایی ما می‌شود.

در کد پایین ما با دستور constraint.pi بدست آوردیم که با تغییر یک واحد در پنج محدودیت اول ما که همان محدودیت‎های تقاضا هستند(در کامنت گذاری هم نوشته شده) objective function ما چه تغییری میکند 

In [24]:
# Solve the problem
model.solve()

# Print the results and the solving status
print(f"Status: {LpStatus[model.status]}, Objective Value: {value(model.objective)}")

# Print dual values (shadow prices) for the first 5 demand constraints
constraint_count = 0
for constraint_expr in model.constraints.keys():
    constraint_count += 1
    print(f"Marginal cost of producing electricity in interval {constraint_count} is {model.constraints[constraint_expr].pi}")
    if constraint_count == 5:
        break


Status: Optimal, Objective Value: 580200.0
Marginal cost of producing electricity in interval 1 is 2.0
Marginal cost of producing electricity in interval 2 is 3.0
Marginal cost of producing electricity in interval 3 is 2.0
Marginal cost of producing electricity in interval 4 is 3.0
Marginal cost of producing electricity in interval 5 is 2.0


### سوال :  کاهش ضمانت خروجی ذخیره 15 درصدی چه صرفه جویی‌ای می کند؟ به بیان دیگر، این تضمین امنیت عرضه، چقدر هزینه دارد؟

منظور از این سوال این بود که اگر ما کلا به فکر افزایش ظرفیت نبودیم هزینه‌ی بهینه‌ی ما برای تامین برق چقدر بود.

بدین منظور محدودیت‌های مربوط به افزایش 15 درصدی ظرفیت را پاک میکنیم(constraint 3: Ability to increase load up to 15%). همجنین در تابع هدف قسمت مربوطه به هزینه‌ی اضافی بر خداقل سطح برای هر مگاوات ساعت را حذف میکنیم.

جواب نهایی هزینه‌ی حداقل در حالتی است که ما به فکر 15 درصد افزایش سطح برق نباشیم.

پس عملا هزینه‌ی اضافیی که ما برای افزایش سطح می‌کنیم حاصل کم کردن objective value مدل 1 و مدل زیر است

In [25]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus, LpMaximize
import pandas as pd

# Define the problem
model_ = LpProblem(name="GeneratorOptimization", sense=LpMinimize)

# define the number of j,K,L,M's for our varaibles
j_intervals = range(1, 6)
K_generators = range(1, 13)
L_generators = range(1, 11)
M_generators = range(1, 6)
# Define decision variables
S = LpVariable.dicts("S", ((t, j) for t in range(1, 4) for j in j_intervals), lowBound=0, cat="Integer")
x_K = LpVariable.dicts("xK", ((k, j) for k in K_generators for j in j_intervals), lowBound=0, cat="Continuous")
x_L = LpVariable.dicts("xL", ((l, j) for l in L_generators for j in j_intervals), lowBound=0, cat="Continuous")
x_M = LpVariable.dicts("xM", ((m, j) for m in M_generators for j in j_intervals), lowBound=0, cat="Continuous")
Y_K = LpVariable.dicts("YK", ((K, j) for K in K_generators for j in j_intervals), cat="Binary")
Y_L = LpVariable.dicts("YL", ((L, j) for L in L_generators for j in j_intervals), cat="Binary")
Y_M = LpVariable.dicts("YM", ((M, j) for M in M_generators for j in j_intervals), cat="Binary")
T = {1: 6, 2: 3, 3: 6, 4: 3, 5: 6}

# Define objective function from the model 1 of phase1 without increase load up to 15%
model_ +=  lpSum(
    1000 * (lpSum(Y_K[K, j]*T[j] for K in K_generators)) +
    2600 * (lpSum(Y_L[L, j]*T[j] for L in L_generators)) +
    3000 * (lpSum(Y_M[M, j]*T[j] for M in M_generators)) for j in j_intervals   
) + lpSum(2000 * S[1, j] + 1000 * S[2, j] + 500 * S[3, j] for j in j_intervals)

## Define constraints

# constraint 1: Demand constraints
model_ += lpSum(x_K[K, 1] for K in K_generators )+ lpSum(x_L[L, 1] for L in L_generators )+lpSum(x_M[M, 1] for M in M_generators ) >= 15000
model_ += lpSum(x_K[K, 2] for K in K_generators )+ lpSum(x_L[L, 2] for L in L_generators )+lpSum(x_M[M, 2] for M in M_generators ) >= 30000
model_ += lpSum(x_K[K, 3] for K in K_generators )+ lpSum(x_L[L, 3] for L in L_generators )+lpSum(x_M[M, 3] for M in M_generators ) >= 25000
model_ += lpSum(x_K[K, 4] for K in K_generators )+ lpSum(x_L[L, 4] for L in L_generators )+lpSum(x_M[M, 4] for M in M_generators ) >= 40000
model_ += lpSum(x_K[K, 5] for K in K_generators )+ lpSum(x_L[L, 5] for L in L_generators )+lpSum(x_M[M, 5] for M in M_generators ) >= 27000

# constraint 2: Minimum and maximum range constraints
for j in j_intervals:
    for K in K_generators:
        model_ += x_K[K, j] >= 850 * Y_K[K, j] 
        model_ += x_K[K, j] <= 2000 * Y_K[K, j]

    for L in L_generators:
        model_ += x_L[L, j] >= 1250 * Y_L[L, j] 
        model_ += x_L[L, j] <= 1750 * Y_L[L, j]

    for M in M_generators:
        model_ += x_M[M, j] >= 1500 * Y_M[M, j] 
        model_ += x_M[M, j] <= 4000 * Y_M[M, j]
        
        
# # constraint 3: Ability to increase load up to 15%
# model += 2000 * lpSum(Y_K[K, 1] for K in K_generators)+ 1750 * lpSum(Y_L[L, 1] for L in L_generators) +4000*lpSum(Y_M[M, 1] for M in M_generators) >= 1.15 * 15000
# model += 2000 * lpSum(Y_K[K, 2] for K in K_generators)+ 1750 * lpSum(Y_L[L, 2] for L in L_generators) +4000*lpSum(Y_M[M, 2] for M in M_generators) >= 1.15 * 30000
# model += 2000 * lpSum(Y_K[K, 3] for K in K_generators)+ 1750 * lpSum(Y_L[L, 3] for L in L_generators) +4000*lpSum(Y_M[M, 3] for M in M_generators) >= 1.15 * 25000
# model += 2000 * lpSum(Y_K[K, 4] for K in K_generators)+ 1750 * lpSum(Y_L[L, 4] for L in L_generators) +4000*lpSum(Y_M[M, 4] for M in M_generators) >= 1.15 * 40000
# model += 2000 * lpSum(Y_K[K, 5] for K in K_generators)+ 1750 * lpSum(Y_L[L, 5] for L in L_generators) +4000*lpSum(Y_M[M, 5] for M in M_generators) >= 1.15 * 27000

# constraint 4: The limitation of relationship between the generator start-up variable and the number of working units
for j in j_intervals:
    if j == 1:
        model_ += S[1, j] >= lpSum(Y_K[K, j] for K in K_generators) - lpSum(Y_K[K, 5] for K in K_generators)
        model_ += S[2, j] >= lpSum(Y_L[L, j] for L in L_generators) - lpSum(Y_L[L, 5] for L in L_generators)
        model_ += S[3, j] >= lpSum(Y_M[M, j] for M in M_generators) - lpSum(Y_M[M, 5] for M in M_generators)
    else:
        model_ += S[1, j] >= lpSum(Y_K[K, j] for K in K_generators) - lpSum(Y_K[K, j-1] for K in K_generators)
        model_ += S[2, j] >= lpSum(Y_L[L, j] for L in L_generators) - lpSum(Y_L[L, j-1] for L in L_generators)
        model_ += S[3, j] >= lpSum(Y_M[M, j] for M in M_generators) - lpSum(Y_M[M, j-1] for M in M_generators)

# Solve the problem
model_.solve()

# Print the results and the solving status
print(f"Status: {LpStatus[model_.status]}, Objective Value: {value(model_.objective)}")

# The additional cost to increase the level
print(f"The additional cost to increase the electricity level is {value(model.objective) - value(model_.objective)} ")


Status: Optimal, Objective Value: 355500.0
The additional cost to increase the electricity level is 224700.0 


### سوال 2 :

نام گذاری تمام متغیرها بر اساس نام گذاری فاز اول انجام شده و بر همان اساس است.

بدین صورت که x_K,x_L,x_M سطح خروجی ژنراتور K,L,Mام نوع 1و2و3 در بازه زمانی j هستند.(K نوع اول، L نوع دوم و M نوع سوم)

متغیرهای Y_K,Y_L,Y_M هم نشان می‌دهند در بازه j ژنراتور مربوطه کار میکند یا خیر.

متغیر g نشان میدهد که ژنراتور آبی نوع i در بازه j کار میکند یا نه( دو بعد i,j)

متغیر z هم نشان میدهد که ژنراتور آبی نوع i در بازه j شروع به کار میکند یا نه( دو بعد i , j)

متغیر l عمق مخزن در ابتدای بازه j را نشان مشدهد(یک بعد j)

متغیر p میزان مگاوات برقی که برای افزایش عمق مخزن در بازه j استفاده می شود (یک بعد j)

و T تعداد ساعت در بازه زمانی j را نشان میدهد که ورودی مساله است

و R میزان کاهش عمق در هر ساعت توسط ژنراتور آبی نوع i است که ورودی مساله است

In [26]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus
import pandas as pd


# Define the problem
model2 = LpProblem(name="GeneratorOptimization2", sense=LpMinimize)

# define the number of j,i,K,L,M's for our varaibles
j_intervals = range(1, 6)
i_intervals = ["A" , "B"]
K_generators = range(1, 13)
L_generators = range(1, 11)
M_generators = range(1, 6)

# Define decision variables
g=LpVariable.dicts("g", ((i, j) for i in i_intervals for j in j_intervals), cat="Binary")
z=LpVariable.dicts("z", ((i, j) for i in i_intervals for j in j_intervals), cat="Binary")
l=LpVariable.dicts("l", ( j for j in j_intervals), lowBound=0, cat="Continuous")
p=LpVariable.dicts("p", ( j for j in j_intervals), lowBound=0, cat="Continuous")
S_ = LpVariable.dicts("S", ((i, j) for i in range(1, 4) for j in j_intervals), lowBound=0, cat="Integer")
x_K_ = LpVariable.dicts("xK", ((k, j) for k in K_generators for j in j_intervals), lowBound=0, cat="Continuous")
x_L_ = LpVariable.dicts("xL", ((l, j) for l in L_generators for j in j_intervals), lowBound=0, cat="Continuous")
x_M_ = LpVariable.dicts("xM", ((m, j) for m in M_generators for j in j_intervals), lowBound=0, cat="Continuous")
Y_K_ = LpVariable.dicts("YK", ((k, j) for k in K_generators for j in j_intervals), cat="Binary")
Y_L_ = LpVariable.dicts("YL", ((l, j) for l in L_generators for j in j_intervals), cat="Binary")
Y_M_ = LpVariable.dicts("YM", ((m, j) for m in M_generators for j in j_intervals), cat="Binary")
T = {1: 6, 2: 3, 3: 6, 4: 3, 5: 6}
R = {'A': 0.31, 'B': 0.47}

# Define objective function from the model 2 of phase1
model2 += lpSum(
    2 * (lpSum(x_K_[k, j] for k in K_generators for j in j_intervals) - 850 * lpSum(Y_K_[K, j] for K in K_generators for j in j_intervals)) +
    1.3 * (lpSum(x_L_[l, j] for l in L_generators for j in j_intervals) - 1250 * lpSum(Y_L_[L, j] for L in L_generators for j in j_intervals)) +
    3 * (lpSum(x_M_[m, j] for m in M_generators for j in j_intervals) - 1500 * lpSum(Y_M_[M, j] for M in M_generators for j in j_intervals))
) + lpSum(
    1000 *lpSum(Y_K_[K, j]*T[j] for K in K_generators for j in j_intervals) +
    2600 * lpSum(Y_L_[L, j]*T[j] for L in L_generators for j in j_intervals ) +
    3000 * lpSum(Y_M_[M, j]*T[j] for M in M_generators for j in j_intervals)
) + lpSum(2000 * S_[1, j] + 1000 * S_[2, j] + 500 * S_[3, j] for j in j_intervals)
+ lpSum(1500 * z["A",j] for j in j_intervals) +lpSum(1200 * z["B",j] for j in j_intervals) 
+ lpSum(90 * g["A",j]*T[j] for j in j_intervals) + lpSum(150 * g["B",j]*T[j] for j in j_intervals)


## Define constraints

# constraint 1: Demand constraints
model2 += lpSum(x_K_[K, 1] for K in K_generators )+ lpSum(x_L_[L, 1] for L in L_generators )+lpSum(x_M_[M, 1] for M in M_generators )+900*g["A",1] +1400*g["B",1]-p[1] >= 15000
model2 += lpSum(x_K_[K, 2] for K in K_generators )+ lpSum(x_L_[L, 2] for L in L_generators )+lpSum(x_M_[M, 2] for M in M_generators )+900*g["A",2] +1400*g["B",2]-p[2] >= 30000
model2 += lpSum(x_K_[K, 3] for K in K_generators )+ lpSum(x_L_[L, 3] for L in L_generators )+lpSum(x_M_[M, 3] for M in M_generators )+900*g["A",3] +1400*g["B",3]-p[3] >= 25000
model2 += lpSum(x_K_[K, 4] for K in K_generators )+ lpSum(x_L_[L, 4] for L in L_generators )+lpSum(x_M_[M, 4] for M in M_generators )+900*g["A",4] +1400*g["B",4]-p[4] >= 40000
model2 += lpSum(x_K_[K, 5] for K in K_generators )+ lpSum(x_L_[L, 5] for L in L_generators )+lpSum(x_M_[M, 5] for M in M_generators )+900*g["A",5] +1400*g["B",5]-p[5] >= 27000

# constraint 2: Minimum and maximum range constraints
for j in j_intervals:
    for k in K_generators:
        if (k, j) in Y_K_:
            model2 += 850 * Y_K_[k, j] <= x_K_[k, j] 
            model2 += x_K_[k, j]  <= 2000 * Y_K_[k, j]

    for L in L_generators:
        if (L, j) in Y_L_:
            model2 += 1250 * Y_L_[L, j] <= x_L_[L, j] 
            model2 += x_L_[L, j] <= 1750 * Y_L_[L, j]

    for m in M_generators:
        if (m, j) in Y_M_:
            model2 += 1500 * Y_M_[m, j] <= x_M_[m, j] 
            model2 += x_M_[m, j] <= 4000 * Y_M_[m, j]
        
        
# constraint 3: Depth limit  
# *** note that l[1] is 16
model2 += l[1]==16
for j in j_intervals:       
    model2 += l[j] + ((T[j] * p[j]) / 3000) - lpSum(T[j] * R[i] * g[i, j] for i in i_intervals) >= 15
    model2 += l[j] + ((T[j] * p[j]) / 3000) - lpSum(T[j] * R[i] * g[i, j] for i in i_intervals) <= 20

        
        
# constraint 4: Ability to increase load up to 15%
model2 += 2000 * lpSum(Y_K_[K, 1] for K in K_generators)+ 1750 * lpSum(Y_L_[L, 1] for L in L_generators) +4000*lpSum(Y_M_[M, 1] for M in M_generators) +900 +1400>= 1.15 * 15000
model2 += 2000 * lpSum(Y_K_[K, 2] for K in K_generators)+ 1750 * lpSum(Y_L_[L, 2] for L in L_generators) +4000*lpSum(Y_M_[M, 2] for M in M_generators) +900 +1400>= 1.15 * 30000
model2 += 2000 * lpSum(Y_K_[K, 3] for K in K_generators)+ 1750 * lpSum(Y_L_[L, 3] for L in L_generators) +4000*lpSum(Y_M_[M, 3] for M in M_generators) +900 +1400>= 1.15 * 25000
model2 += 2000 * lpSum(Y_K_[K, 4] for K in K_generators)+ 1750 * lpSum(Y_L_[L, 4] for L in L_generators) +4000*lpSum(Y_M_[M, 4] for M in M_generators) +900 +1400>= 1.15 * 40000
model2 += 2000 * lpSum(Y_K_[K, 5] for K in K_generators)+ 1750 * lpSum(Y_L_[L, 5] for L in L_generators) +4000*lpSum(Y_M_[M, 5] for M in M_generators) +900 +1400>= 1.15 * 27000



# constraint 5: The limitation of communication between the start-up and the number of working units
for j in j_intervals:
    if j == 1:
        model2 += S_[1, j] >= lpSum(Y_K_[K, j] - Y_K_[K, 5] for K in K_generators)
        model2 += S_[2, j] >= lpSum(Y_L_[L, j] - Y_L_[L, 5] for L in L_generators)
        model2 += S_[3, j] >= lpSum(Y_M_[M, j] - Y_M_[M, 5] for M in M_generators)
    else:
        model2 += S_[1, j] >= lpSum(Y_K_[K, j] - Y_K_[K, j-1] for K in K_generators)
        model2 += S_[2, j] >= lpSum(Y_L_[L, j] - Y_L_[L, j-1] for L in L_generators)
        model2 += S_[3, j] >= lpSum(Y_M_[M, j] - Y_M_[M, j-1] for M in M_generators)

for j in j_intervals:
    for i in i_intervals:
        if j==1 :
            model2 += z[i,1]>=g[i,1]-g[i,5]
        else:
            model2 += z[i,j]>=g[i,j]-g[i,j-1]
            
            
# Solve the problem
model2.solve()

# Print the results
print(f"Status: {LpStatus[model2.status]}, Objective Value: {value(model2.objective)}")
for v in model2.variables():
    # Save values to the corresponding dataframe
    print(f"{v.name}: {v.varValue}")



Status: Optimal, Objective Value: 529270.0
S_(1,_1): 0.0
S_(1,_2): 4.0
S_(1,_3): 0.0
S_(1,_4): 0.0
S_(1,_5): 0.0
S_(2,_1): 0.0
S_(2,_2): 1.0
S_(2,_3): 0.0
S_(2,_4): 0.0
S_(2,_5): 0.0
S_(3,_1): 0.0
S_(3,_2): 2.0
S_(3,_3): 0.0
S_(3,_4): 4.0
S_(3,_5): 0.0
YK_(1,_1): 1.0
YK_(1,_2): 1.0
YK_(1,_3): 1.0
YK_(1,_4): 1.0
YK_(1,_5): 1.0
YK_(10,_1): 0.0
YK_(10,_2): 1.0
YK_(10,_3): 1.0
YK_(10,_4): 1.0
YK_(10,_5): 1.0
YK_(11,_1): 1.0
YK_(11,_2): 1.0
YK_(11,_3): 1.0
YK_(11,_4): 1.0
YK_(11,_5): 0.0
YK_(12,_1): 1.0
YK_(12,_2): 1.0
YK_(12,_3): 1.0
YK_(12,_4): 1.0
YK_(12,_5): 1.0
YK_(2,_1): 1.0
YK_(2,_2): 1.0
YK_(2,_3): 1.0
YK_(2,_4): 1.0
YK_(2,_5): 1.0
YK_(3,_1): 0.0
YK_(3,_2): 1.0
YK_(3,_3): 1.0
YK_(3,_4): 1.0
YK_(3,_5): 1.0
YK_(4,_1): 1.0
YK_(4,_2): 1.0
YK_(4,_3): 1.0
YK_(4,_4): 1.0
YK_(4,_5): 1.0
YK_(5,_1): 0.0
YK_(5,_2): 1.0
YK_(5,_3): 1.0
YK_(5,_4): 1.0
YK_(5,_5): 1.0
YK_(6,_1): 1.0
YK_(6,_2): 1.0
YK_(6,_3): 1.0
YK_(6,_4): 1.0
YK_(6,_5): 1.0
YK_(7,_1): 1.0
YK_(7,_2): 1.0
YK_(7,_3): 1.0
YK_(7,_4): 1

In [27]:
# Create an empty dataframe with the desired column names
df_x_2 = pd.DataFrame(columns=["K generator", "L generator", "M generator"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    row_values_2 = {
        "K generator": sum([x_K_[k, j].varValue for k in K_generators]),
        "L generator": sum([x_L_[l, j].varValue for l in L_generators]),
        "M generator": sum([x_M_[m, j].varValue for m in M_generators])
    }
    df_x_2 = df_x_2.append(row_values_2, ignore_index=True)
# Set the row index to start from 1
df_x_2.index = range(1, len(df_x_2) + 1)
    
# Display the final dataframe
print("The x_K and x_L and x_M in 5 intervals")
print()
print(df_x_2)


The x_K and x_L and x_M in 5 intervals

   K generator  L generator  M generator
1      14510.0          0.0          0.0
2      22950.0       1750.0       3000.0
3      21200.0          0.0       1500.0
4      24000.0          0.0      13700.0
5      21700.0          0.0       3000.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1761469211.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x_2 = df_x_2.append(row_values_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1761469211.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x_2 = df_x_2.append(row_values_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1761469211.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x_2 = df_x_2.append(row_values_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1761469211.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_x_2 = df_x_2.append(r

In [28]:
# Create an empty dataframe with the desired column names
df_y_2 = pd.DataFrame(columns=["K generator", "L generator", "M generator"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    row_values_y_2 = {
        "K generator": sum([Y_K_[k, j].varValue for k in K_generators]),
        "L generator": sum([Y_L_[l, j].varValue for l in L_generators]),
        "M generator": sum([Y_M_[m, j].varValue for m in M_generators])
    }
    df_y_2 = df_y_2.append(row_values_y_2, ignore_index=True)
# Set the row index to start from 1
df_y_2.index = range(1, len(df_y_2) + 1)
    
# Display the final dataframe
print("The x_K and x_L and x_M in 5 intervals")
print()
print(df_y_2)


The x_K and x_L and x_M in 5 intervals

   K generator  L generator  M generator
1          8.0          0.0          0.0
2         12.0          1.0          2.0
3         12.0          0.0          1.0
4         12.0          0.0          5.0
5         11.0          0.0          2.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1280805584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y_2 = df_y_2.append(row_values_y_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1280805584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y_2 = df_y_2.append(row_values_y_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1280805584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y_2 = df_y_2.append(row_values_y_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1280805584.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_y_2 = df_y_2.ap

In [29]:
# Create an empty dataframe with the desired column names
df_S_2 = pd.DataFrame(columns=["K generator", "L generator", "M generator"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    for t in range(1,4):
        row_values_S_2 = {
            "K generator": S_[t, j].varValue ,
            "L generator": S_[t, j].varValue ,
            "M generator": S_[t, j].varValue
    }
    df_S_2 = df_S_2.append(row_values_S_2, ignore_index=True)
# Set the row index to start from 1
df_S_2.index = range(1, len(df_S_2) + 1)
    
# Display the final dataframe
print("The S for 3 generator types and 5 intervals")
print()
print(df_S_2)


The S for 3 generator types and 5 intervals

   K generator  L generator  M generator
1          0.0          0.0          0.0
2          2.0          2.0          2.0
3          0.0          0.0          0.0
4          4.0          4.0          4.0
5          0.0          0.0          0.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\438783681.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S_2 = df_S_2.append(row_values_S_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\438783681.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S_2 = df_S_2.append(row_values_S_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\438783681.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S_2 = df_S_2.append(row_values_S_2, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\438783681.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_S_2 = df_S_2.append

In [30]:
# Create an empty dataframe with the desired column names
df_g = pd.DataFrame(columns=["type A", "type B"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    for i in i_intervals:
        row_values_g = {
            "type A": g[i, j].varValue ,
            "type B": g[i, j].varValue 
    }
    df_g = df_g.append(row_values_g, ignore_index=True)
# Set the row index to start from 1
df_g.index = range(1, len(df_g) + 1)
    
# Display the final dataframe
print("The g for 2 hydro generators and 5 intervals")
print()
print(df_g)


The g for 2 hydro generators and 5 intervals

   type A  type B
1     1.0     1.0
2     1.0     1.0
3     1.0     1.0
4     1.0     1.0
5     1.0     1.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1215263883.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_g = df_g.append(row_values_g, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1215263883.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_g = df_g.append(row_values_g, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1215263883.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_g = df_g.append(row_values_g, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1215263883.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_g = df_g.append(row_values_g, ign

In [31]:
# Create an empty dataframe with the desired column names
df_z = pd.DataFrame(columns=["type A", "type B"])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    for i in i_intervals:
        row_values_z = {
            "type A": z[i, j].varValue ,
            "type B": z[i, j].varValue 
    }
    df_z = df_z.append(row_values_z, ignore_index=True)
# Set the row index to start from 1
df_z.index = range(1, len(df_z) + 1)
    
# Display the final dataframe
print("The z for 2 hydro generators and 5 intervals")
print()
print(df_z)


The z for 2 hydro generators and 5 intervals

   type A  type B
1     1.0     1.0
2     1.0     1.0
3     1.0     1.0
4     1.0     1.0
5     1.0     1.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\4173756129.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_z = df_z.append(row_values_z, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\4173756129.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_z = df_z.append(row_values_z, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\4173756129.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_z = df_z.append(row_values_z, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\4173756129.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_z = df_z.append(row_values_z, ign

In [32]:
# Create an empty dataframe with the desired column names
df_l = pd.DataFrame(columns=['value'])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    row_values_l = {
        "value": l[j].varValue 
    }
    df_l = df_l.append(row_values_l, ignore_index=True)
# Set the row index to start from 1
df_l.index = range(1, len(df_l) + 1)
    
# Display the final dataframe
print("The l for 5 intervals")
print()
print(df_l)


The l for 5 intervals

   value
1  16.00
2  22.34
3  24.68
4  22.34
5  24.68


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1224031402.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_l = df_l.append(row_values_l, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1224031402.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_l = df_l.append(row_values_l, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1224031402.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_l = df_l.append(row_values_l, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1224031402.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_l = df_l.append(row_values_l, ignore_

In [33]:
# Create an empty dataframe with the desired column names
df_p = pd.DataFrame(columns=['value'])

# Iterate over the j_intervals and save the values to the dataframe
for j in range(1,len(j_intervals)+1):
    row_values_p = {
        "value": p[j].varValue 
    }
    df_p = df_p.append(row_values_p, ignore_index=True)
# Set the row index to start from 1
df_p.index = range(1, len(df_p) + 1)
    
# Display the final dataframe
print("The p for 5 intervals")
print()
print(df_p)


The p for 5 intervals

   value
1  910.0
2    0.0
3    0.0
4    0.0
5    0.0


C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1832708026.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_p = df_p.append(row_values_p, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1832708026.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_p = df_p.append(row_values_p, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1832708026.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_p = df_p.append(row_values_p, ignore_index=True)
C:\Users\hs138\AppData\Local\Temp\ipykernel_3220\1832708026.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_p = df_p.append(row_values_p, ignore_